### Introduction to Blockchain

In [ ]:
from time import localtime, strftime
from typing import List

import hashlib
import json

In [ ]:
class Transaction:
    def __init__(self, sender: str, receiver: str, amount: float):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount

    def __repr__(self):
        return f'{self.sender} -> {self.receiver}: {self.amount}'

class Block:
    def __init__(self, transactions: List[Transaction], previous_hash: str):
        self.timestamp = strftime('%Y-%m-%d %H:%M:%S', localtime())
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        tx_str = ''.join(str(tx) for tx in self.transactions)
        block_string = f'{self.timestamp}{tx_str}{self.previous_hash}'
        return hashlib.sha256(block_string.encode()).hexdigest()

    def __repr__(self):
        return json.dumps({
            'hash': self.hash,
            'previous_hash': self.previous_hash,
            'transactions': [str(tx) for tx in self.transactions],
            'timestamp': self.timestamp
        }, indent=4)

class Blockchain:
    def __init__(self):
        self.chain: List[Block] = [self.create_genesis_block()]
        self.pending_transactions: List[Transaction] = []

    def create_genesis_block(self):
        return Block(transactions=[], previous_hash='0')

    def get_latest_block(self):
        return self.chain[-1]

    def add_transaction(self, transaction: Transaction):
        self.pending_transactions.append(transaction)

    def mine_block(self):
        if not self.pending_transactions:
            return None

        new_block = Block(
            transactions=self.pending_transactions,
            previous_hash=self.get_latest_block().hash
        )
        self.chain.append(new_block)
        self.pending_transactions = []
        return new_block

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            curr = self.chain[i]
            prev = self.chain[i - 1]
            if curr.hash != curr.calculate_hash():
                return False
            if curr.previous_hash != prev.hash:
                return False
        return True

    def __repr__(self):
        return '\n'.join(str(block) for block in self.chain)

In [ ]:
blockchain = Blockchain()
print(f"Blockchain: {blockchain}")

In [ ]:
blockchain.add_transaction(Transaction('Alice', 'Eve', 50))
mined_block = blockchain.mine_block()
print(f"Blockchain: {blockchain}")
print('\n# -------------------------\n')
print(f"Mined Block 1: {mined_block}")

In [ ]:
blockchain.add_transaction(Transaction('Bob', 'Alice', 25))
mined_block = blockchain.mine_block()
print(f"Blockchain: {blockchain}")
print('\n# -------------------------\n')
print(f"Mined Block 2: {mined_block}")

In [ ]:
print(f"Validity: {blockchain.is_chain_valid()}")
print('\n# -------------------------\n')

# -------------------------

blockchain.chain[1].transactions[0].amount = 1000
print(f"Tampered Block 1: {blockchain.chain[1]}")

# -------------------------

print('\n# -------------------------\n')
print(f"Validity: {blockchain.is_chain_valid()}")